In [1]:
## run for the first time
# import nltk; nltk.download('stopwords')

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'idea', 'proposal'])

In [3]:
# Import Dataset
df = pd.read_json('data_ccrit.json')
df.head()

,idea,desctiption,task,comment,reaction
0,Build a tower,Build a tower in the center of El Nudillo. Ske...,Comment on the Idea,A structure or monument at the Nudillo could i...,neutral
1,Build a tower,Build a tower in the center of El Nudillo. Ske...,Comment on the Idea,The built form should be lit at night.,neutral
2,Build a tower,Build a tower in the center of El Nudillo. Ske...,Comment on the Idea,"More than a plaza, the space can be activated ...",excited
3,Build a tower,Build a tower in the center of El Nudillo. Ske...,Comment on the Idea,The structure could incorporate an observation...,excited
4,Build a tower,Build a tower in the center of El Nudillo. Ske...,Comment on the Idea,I think its a great idea,excited


In [5]:
# get proposal wise comments
ideas = df.idea.unique()
proposal_wise_comments = []

for idea in ideas:
    proposal_wise_comments.append(list(df.loc[df['idea'] == idea, 'comment']))

19

In [5]:
### Preprocessing

# cleanup
def cleanup(data):
    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
    
    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]
    
    return data

# tokenize
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


def create_gram_models(data_words):
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

#     # See trigram example
#     print(trigram_mod[bigram_mod[data_words[0]]])
    
    return [bigram_mod, trigram_mod]

# functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [6]:
# model generation

def create_dict_and_corpus(data_lemmatized):
    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    # # Human readable format of corpus (term-frequency)
    # [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]
    
    return id2word, corpus

# create lda model (mallet)
def create_mallet_lda_model(id2word, corpus):
    # # Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
    import os
    os.environ.update({'MALLET_HOME': r'/home/mjasim/Matryona/topic_modeling/mallet/'})
    mallet_path = '/home/mjasim/Matryona/topic_modeling/mallet/bin/mallet'
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)
    
    return ldamallet

In [7]:
# Compute Coherence Score
def get_coherence_score(ldamallet, data_lemmatized, id2word):
    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    return coherence_ldamallet

In [8]:
# get optimal number of topics

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    mallet_path = '/home/mjasim/Matryona/topic_modeling/mallet/bin/mallet'
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)        
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [9]:
# find the best model
def get_best_model(model_list, coherence_values, start, limit, step):
    max = 0
    num_topics = 0
    list_idx = 0
    result = []
    x = range(start, limit, step)
    for m, cv in zip(x, coherence_values):
#         print("Num Topics =", m, " has Coherence Value of", round(cv, 4))
        if(cv > max):
            max = cv
            num_topics = m
            model_idx = list_idx
        list_idx += 1

    # Select the best model
    optimal_model = model_list[model_idx]
    
    return optimal_model

In [25]:
### find dominant topic in each sentence

def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()
    
    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [33]:
# pipeline

frames = []

for data in proposal_wise_comments:
    data_words = cleanup(data)        
    data_words = list(sent_to_words(data))
    bigram_mod, trigram_mod = create_gram_models(data_words)
    data_words_nostops = remove_stopwords(data_words)
    data_words_bigrams = make_bigrams(data_words_nostops)
    nlp = spacy.load('en', disable=['parser', 'ner'])
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    id2word, corpus = create_dict_and_corpus(data_lemmatized)
    ldamallet = create_mallet_lda_model(id2word, corpus)
    coherence_ldamallet = get_coherence_score(ldamallet, data_lemmatized, id2word)
    start = 2
    limit = 30
    step = 2
    model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=start, limit=limit, step=step)
    optimal_model = get_best_model(model_list, coherence_values, start, limit, step)
    df_topic_sents_keywords = format_topics_sentences(optimal_model, corpus, data)
    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    frames.append(df_dominant_topic)

sent_wise_df = pd.concat(frames)

/home/mjasim/environments/env3.6/lib/python3.6/site-packages/gensim/models/wrappers/ldamallet.py:379: RuntimeWarning: invalid value encountered in true_divide
  return topics / topics.sum(axis=1)[:, None]
/home/mjasim/environments/env3.6/lib/python3.6/site-packages/gensim/models/wrappers/ldamallet.py:448: RuntimeWarning: invalid value encountered in true_divide
  topic = topic / topic.sum()  # normalize to probability dist


In [35]:
sent_wise_df

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0.0,0.1879,"city, village, chime, sand, landmark, shopper,...",A structure or monument at the Nudillo could i...
1,1,5.0,0.1749,"mobility, impact, incorporate, insular, amazin...",The built form should be lit at night.
2,2,4.0,0.2037,"place, people, safety, special, draw, dense, r...","More than a plaza, the space can be activated ..."
3,3,5.0,0.1984,"mobility, impact, incorporate, insular, amazin...",The structure could incorporate an observation...
4,4,0.0,0.1786,"city, village, chime, sand, landmark, shopper,...",I think its a great idea
...,...,...,...,...,...
18,18,0.0,0.1394,"public, live, possibility, encourage, remove, ...","Depends on the location, and the activity of t..."
19,19,0.0,0.1250,"public, live, possibility, encourage, remove, ...",not sure
20,20,2.0,0.1672,"market, attracted, sense, barrio, craft, inter...",Suggest to work with the East Village Associat...
21,21,5.0,0.1528,"community, great, fair, promoter, critical, li...",This is a great idea to activate the area and ...


In [52]:
final_topics = []
terms = []
for i in sent_wise_df['Keywords']:
    final_topics.append(i.split(',')[0])
    temp = []
    terms.append(temp)
sent_wise_df['terms'] = terms
sent_wise_df['aspect'] = final_topics

new_df = sent_wise_df[['Text', 'terms', 'aspect']].copy()
new_df.rename(columns = {'Text':'text'}, inplace = True) 

In [56]:
new_df.to_json("cc_topic_aspect.json", orient="records")

In [51]:
# final_topics = []
# for i in all_topics:
#     topic_keywords = []
#     ft = list(set(i))
#     topic_keywords = ", ".join([word for word in ft])
#     final_topics.append(topic_keywords)

In [ ]:
# dictofWords = { i : final_topics[i] for i in range(0, len(final_topics) ) }

# dictofWords

In [40]:
# df_topics = pd.DataFrame.from_dict(dictofWords, orient='index')

# df_topics.columns = ["topic_keywords"]

# df_topics.to_json()

In [58]:
xxx = pd.read_json("cc_topic_aspect.json")

In [59]:
xxx

,text,terms,aspect
0,A structure or monument at the Nudillo could i...,[],city
1,The built form should be lit at night.,[],mobility
2,"More than a plaza, the space can be activated ...",[],place
3,The structure could incorporate an observation...,[],mobility
4,I think its a great idea,[],city
...,...,...,...
339,"Depends on the location, and the activity of t...",[],public
340,not sure,[],public
341,Suggest to work with the East Village Associat...,[],market
342,This is a great idea to activate the area and ...,[],community
